In [18]:
import pandas as pd
import dask.dataframe as dd
import geopandas as gpd

In [19]:
file_path = 'pp-2023.csv'
# Define column names
column_names = ['transaction_id','price', 'transfer_date', 'postcode', 'property_type', 'new_build_flag', 'duration', 'primary_address', 'secondary_address', 'street', 'locality', 'city', 'district', 'county', 'ppd_category', 'record_status']

# Specify data types for each column
dtypes = dict.fromkeys(column_names, 'string')
dtypes.update({'price': 'int64'})

# Specify datetime columns for parsing
parse_dates = ['transfer_date']

# Define columns we don't want
usecols = [col for col in column_names if col != 'transaction_id']

# Read CSV using Dask with specified data types and parse_dates
ddf = dd.read_csv(file_path, header=None, names=column_names, dtype=dtypes, parse_dates=parse_dates,usecols=usecols)

# Compute to get a Pandas DataFrame
raw_data_df = ddf.compute()


In [20]:
raw_data_df[raw_data_df['postcode'].isnull()]

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,county,ppd_category,record_status
675,615000,2023-01-06,<NA>,O,N,L,GARAGE 3,<NA>,BARBERS PILES,<NA>,POOLE,"BOURNEMOUTH, CHRISTCHURCH AND POOLE","BOURNEMOUTH, CHRISTCHURCH AND POOLE",B,A
2011,27126,2023-06-22,<NA>,O,N,F,PLOT ST03,<NA>,CLAPPER LANE,STAPLEHURST,TONBRIDGE,MAIDSTONE,KENT,B,A
2012,24750,2023-06-19,<NA>,O,N,F,PLOT ST68,<NA>,CLAPPER LANE,STAPLEHURST,TONBRIDGE,MAIDSTONE,KENT,B,A
2013,22500,2023-07-27,<NA>,O,N,F,PLOT ST06,<NA>,CLAPPER LANE,STAPLEHURST,TONBRIDGE,MAIDSTONE,KENT,B,A
2014,25542,2023-07-27,<NA>,O,N,F,PLOT ST02,<NA>,CLAPPER LANE,STAPLEHURST,TONBRIDGE,MAIDSTONE,KENT,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657097,204000,2023-02-21,<NA>,O,N,F,METHODIST CHURCH,<NA>,<NA>,TREVERVA,PENRYN,CORNWALL,CORNWALL,B,A
657253,160000,2023-02-02,<NA>,O,N,F,GARAGE 82,<NA>,NAPIER ROAD,<NA>,SEAHAM,COUNTY DURHAM,COUNTY DURHAM,B,A
657503,200000,2023-01-04,<NA>,O,N,F,AUCKLAND PARK RECLAMATION SITE,<NA>,<NA>,AUCKLAND PARK,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM,B,A
657726,75000,2023-02-28,<NA>,O,N,F,BLAINA UNITED REFORMED CHURCH,<NA>,HIGH STREET,BLAINA,ABERTILLERY,BLAENAU GWENT,BLAENAU GWENT,B,A


In [21]:
# Drop rows with no postcode
clean_data_df = raw_data_df.dropna(subset=['postcode'])

#Exclude property type Other (O)
clean_data_df = clean_data_df.loc[clean_data_df['property_type']!='O']

# Sort by Date
clean_data_df.sort_values(by=['transfer_date'], inplace=True, ignore_index=True)

In [22]:
clean_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625602 entries, 0 to 625601
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   price              625602 non-null  int64         
 1   transfer_date      625602 non-null  datetime64[ns]
 2   postcode           625602 non-null  string        
 3   property_type      625602 non-null  string        
 4   new_build_flag     625602 non-null  string        
 5   duration           625602 non-null  string        
 6   primary_address    625602 non-null  string        
 7   secondary_address  74787 non-null   string        
 8   street             617282 non-null  string        
 9   locality           228127 non-null  string        
 10  city               625602 non-null  string        
 11  district           625602 non-null  string        
 12  county             625602 non-null  string        
 13  ppd_category       625602 non-null  string  

In [23]:
clean_data_df['postcode_sector'] = clean_data_df['postcode'].apply(lambda x: x[:x.find(' ')+ 2])

In [24]:
clean_data_df.head()

,price,transfer_date,postcode,property_type,new_build_flag,duration,primary_address,secondary_address,street,locality,city,district,county,ppd_category,record_status,postcode_sector
0,48000,2023-01-01,NR30 2HQ,F,N,L,1,FLAT 2,BRITANNIA ROAD,<NA>,GREAT YARMOUTH,GREAT YARMOUTH,NORFOLK,A,A,NR30 2
1,450000,2023-01-01,CB24 6UA,T,N,F,3,<NA>,LODGE CLOSE,MILTON,CAMBRIDGE,SOUTH CAMBRIDGESHIRE,CAMBRIDGESHIRE,A,A,CB24 6
2,295000,2023-01-01,BS34 5RF,S,N,F,52,<NA>,SWALLOW DRIVE,PATCHWAY,BRISTOL,SOUTH GLOUCESTERSHIRE,SOUTH GLOUCESTERSHIRE,A,A,BS34 5
3,555000,2023-01-01,HA4 0TL,T,N,F,36,<NA>,WEST MEAD,<NA>,RUISLIP,HILLINGDON,GREATER LONDON,A,A,HA4 0
4,130000,2023-01-01,BL3 4LH,T,N,L,383,<NA>,HULTON LANE,<NA>,BOLTON,BOLTON,GREATER MANCHESTER,A,A,BL3 4


In [25]:
# Load regions by postcodes
regions_df = pd.read_csv('postcode_data/postcode_areas.csv', usecols=['postcode_prefix', 'region'])

# Create a dictionary from the DataFrame
postcode_region_dict = dict(zip(regions_df['postcode_prefix'], regions_df['region']))

# Add postcode prefix column to main df
clean_data_df['postcode_prefix'] = clean_data_df['postcode'].str.extract(r'^([A-Z]+)')

# Map the postcode prefix to the region using the dictionary
clean_data_df['region'] = clean_data_df['postcode_prefix'].map(postcode_region_dict)

# Drop the intermediate 'postcode_prefix' column
clean_data_df.drop(columns=['postcode_prefix'], inplace=True)

#Exclude sales in Scotland as limited data available for this region
clean_data_df = clean_data_df.loc[clean_data_df['region']!='Scotland']

In [26]:
# Extract the month from 'transfer_date'
clean_data_df['month'] = clean_data_df['transfer_date'].dt.month

GeoJSON Processing

In [30]:
shpfile = gpd.read_file('shapefiles/Sectors.shp')
shpfile.to_file('geo_json/all_sectors.geojson', driver='GeoJSON')

In [31]:
# Load the postcode prefixes CSV
postcode_prefixes_df = pd.read_csv('postcode_data/postcode_areas.csv')
# Load the GeoJSON file containing postcode sector data
geojson_path = 'geo_json/all_sectors.geojson'
postcode_sectors_gdf = gpd.read_file(geojson_path)


In [32]:
# Create an empty dictionary to store GeoDataFrames for each region
region_geojson_dict = {}

# Extract postcode prefix from 'name' property
postcode_sectors_gdf['postcode_prefix'] = pd.Series(postcode_sectors_gdf['name']).str.extract(r'^([A-Z]+)')

# Merge with postcode prefixes DataFrame to get regions
merged_df = pd.merge(postcode_sectors_gdf, postcode_prefixes_df, how='left', on='postcode_prefix')


In [33]:
# Iterate over unique regions
for region in merged_df['region'].unique():
    # Filter DataFrame for the current region
    region_df = merged_df[merged_df['region'] == region].copy()
    
    # Create a GeoDataFrame for the current region
    region_gdf = gpd.GeoDataFrame(region_df, geometry='geometry', crs=postcode_sectors_gdf.crs)
    
    # Store the GeoDataFrame in the dictionary under the region key
    region_geojson_dict[region] = region_gdf


In [36]:
for region, region_gdf in region_geojson_dict.items():
    region_geojson_path = f'geo_json/regions/{region}_postcode_sectors.geojson'
    region_gdf.to_file(region_geojson_path, driver='GeoJSON')

Average price for each postcode by month


In [37]:
# Group by region, postcode_sector, and month, then calculate the rounded average price and count the number of sales
grouped_df = clean_data_df.groupby(['region', 'postcode_sector', 'month']).agg({'price': ['mean', 'count']}).reset_index()
grouped_df.columns = ['region', 'postcode_sector', 'month','avg_price', 'volume']

# Round the average price to the nearest thousand and convert to integers
grouped_df['avg_price'] = grouped_df['avg_price'].round(-3).astype(int)

grouped_df.head()

,region,postcode_sector,month,avg_price,volume
0,East England,PE1 1,3,115000,1
1,East England,PE1 1,5,100000,1
2,East England,PE1 1,6,124000,2
3,East England,PE1 1,7,450000,1
4,East England,PE1 1,8,128000,3


In [38]:
# Loop through unique years and save CSV for each year
for month in grouped_df['month'].unique():
    # Filter DataFrame for the current year
    month_df = grouped_df[grouped_df['month'] == month]
    
    # Create the CSV file path for the current year
    csv_file_path = (f'processed_data/average_price_by_month/region_data_{month}.csv')
    
    # Save the filtered DataFrame to CSV
    month_df.to_csv(csv_file_path, index=False)

Average price for each region by month


In [39]:
# Group by month, region, property_type, and calculate rounded average price
region_grouped_df = clean_data_df.groupby(['month', 'region', 'property_type']).agg(avg_price=('price', 'mean')).round({'avg_price': -3}).astype({'avg_price': int}).reset_index()

# Calculate sales volume for each group
region_grouped_df['volume'] = clean_data_df.groupby(['month', 'region', 'property_type']).size().reset_index(name='volume')['volume']

# Save the grouped data to a single CSV file
region_grouped_df.to_csv('processed_data/region_avg_price/region_avg_prices.csv', index=False)

Delta month on month


In [43]:
delta_df = grouped_df.copy()  # Create a new DataFrame to store the results
delta_df['delta'] = grouped_df.groupby(['region', 'postcode_sector'])['avg_price'].pct_change() * 100

# Set the percentage change for the first year to 0
delta_df.loc[delta_df['month'] == delta_df['month'].min(), 'delta'] = 0

# Drop any rows with a null delta
delta_df = delta_df.dropna(subset=['delta'])

# Round values to the nearest integer
delta_df['delta'] = delta_df['delta'].round().astype(int)

# Get list of years from df
unique_months = delta_df['month'].unique()

In [44]:
# Iterate over each year and create a CSV file
for month in unique_months:
    # Filter rows for the current year
    month_df = delta_df[delta_df['month'] == month]

    # Create a CSV file for the current year
    csv_file_path = f'processed_data/avg_price_delta/avg_price_delta_{month}.csv'
    month_df.to_csv(csv_file_path, index=False)